## Text Summarization using T5 model

In [1]:
!pip install transformers sentencepiece

In [2]:
import pandas as pd
import torch
import json
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config

2025-01-30 07:07:04.264523: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-30 07:07:04.461192: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738210024.552935    5946 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738210024.577995    5946 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-30 07:07:04.779412: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

## Download and Load the dataset

Download the dataset: [News Summary](https://www.kaggle.com/datasets/sunnysai12345/news-summary)

Once downloaded upload to colab and then proceed

In [3]:
data = pd.read_csv("./data/news_summary.csv", encoding='iso-8859-1')
data.head()

,author,date,headlines,read_more,text,ctext
0,Chhavi Tyagi,"03 Aug 2017,Thursday",Daman & Diu revokes mandatory Rakshabandhan in...,http://www.hindustantimes.com/india-news/raksh...,The Administration of Union Territory Daman an...,The Daman and Diu administration on Wednesday ...
1,Daisy Mowke,"03 Aug 2017,Thursday",Malaika slams user who trolled her for 'divorc...,http://www.hindustantimes.com/bollywood/malaik...,Malaika Arora slammed an Instagram user who tr...,"From her special numbers to TV?appearances, Bo..."
2,Arshiya Chopra,"03 Aug 2017,Thursday",'Virgin' now corrected to 'Unmarried' in IGIMS...,http://www.hindustantimes.com/patna/bihar-igim...,The Indira Gandhi Institute of Medical Science...,The Indira Gandhi Institute of Medical Science...
3,Sumedha Sehra,"03 Aug 2017,Thursday",Aaj aapne pakad liya: LeT man Dujana before be...,http://indiatoday.intoday.in/story/abu-dujana-...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...
4,Aarushi Maheshwari,"03 Aug 2017,Thursday",Hotel staff to get training to spot signs of s...,http://indiatoday.intoday.in/story/sex-traffic...,Hotels in Maharashtra will train their staff t...,Hotels in Mumbai and other Indian cities are t...


In [4]:
data['text'].iloc[0]

'The Administration of Union Territory Daman and Diu has revoked its order that made it compulsory for women to tie rakhis to their male colleagues on the occasion of Rakshabandhan on August 7. The administration was forced to withdraw the decision within 24 hours of issuing the circular after it received flak from employees and was slammed on social media.'

In [5]:
data['ctext'].iloc[0]

'The Daman and Diu administration on Wednesday withdrew a circular that asked women staff to tie rakhis on male colleagues after the order triggered a backlash from employees and was ripped apart on social media.The union territory?s administration was forced to retreat within 24 hours of issuing the circular that made it compulsory for its staff to celebrate Rakshabandhan at workplace.?It has been decided to celebrate the festival of Rakshabandhan on August 7. In this connection, all offices/ departments shall remain open and celebrate the festival collectively at a suitable time wherein all the lady staff shall tie rakhis to their colleagues,? the order, issued on August 1 by Gurpreet Singh, deputy secretary (personnel), had said.To ensure that no one skipped office, an attendance report was to be sent to the government the next evening.The two notifications ? one mandating the celebration of Rakshabandhan (left) and the other withdrawing the mandate (right) ? were issued by the Dama

In [6]:
data['headlines'].iloc[0]

'Daman & Diu revokes mandatory Rakshabandhan in offices order'

## Tokenization and Model Inference

In [7]:
model_name = 't5-small'
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [8]:
text = data['ctext'].iloc[3]
text

'Lashkar-e-Taiba\'s Kashmir commander Abu Dujana was killed in an encounter in a village in Pulwama district of Jammu and Kashmir earlier this week. Dujana, who had managed to give the security forces a slip several times in the past, carried a bounty of Rs 15 lakh on his head.Reports say that Dujana had come to meet his wife when he was trapped inside a house in Hakripora village. Security officials involved in the encounter tried their best to convince Dujana to surrender but he refused, reports say.According to reports, Dujana rejected call for surrender from an Army officer. The Army had commissioned a local to start a telephonic conversation with Dujana. After initiating the talk, the local villager handed over the phone to the army officer."Kya haal hai? Maine kaha, kya haal hai (How are you. I asked, how are you)?" Dujana is heard asking the officer. The officer replies: "Humara haal chhor Dujana. Surrender kyun nahi kar deta. Tu galat kar rha hai (Why don\'t you surrender? You 

In [9]:
before_summarize = text.strip().replace("\n","")
print("Before text summarization:")
before_summarize

Before text summarization:


'Lashkar-e-Taiba\'s Kashmir commander Abu Dujana was killed in an encounter in a village in Pulwama district of Jammu and Kashmir earlier this week. Dujana, who had managed to give the security forces a slip several times in the past, carried a bounty of Rs 15 lakh on his head.Reports say that Dujana had come to meet his wife when he was trapped inside a house in Hakripora village. Security officials involved in the encounter tried their best to convince Dujana to surrender but he refused, reports say.According to reports, Dujana rejected call for surrender from an Army officer. The Army had commissioned a local to start a telephonic conversation with Dujana. After initiating the talk, the local villager handed over the phone to the army officer."Kya haal hai? Maine kaha, kya haal hai (How are you. I asked, how are you)?" Dujana is heard asking the officer. The officer replies: "Humara haal chhor Dujana. Surrender kyun nahi kar deta. Tu galat kar rha hai (Why don\'t you surrender? You 

In [10]:
tokenized_text = tokenizer.encode(before_summarize, return_tensors="pt")

Token indices sequence length is longer than the specified maximum sequence length for this model (735 > 512). Running this sequence through the model will result in indexing errors


In [11]:
summary_ids = model.generate(tokenized_text,
                                    no_repeat_ngram_size=3,
                                    min_length=40,
                                    max_length=150,
                                    early_stopping=True)

/home/teng/development/mlenv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:676: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


In [12]:
summary_ids

tensor([[    0,     3,     9,     3,  1931,  9621,   447,  3634,    28,     8,
          9102,  5502,    47,   708,    28,     3,     9,   415,     3,     5,
             8,  5502, 18606,    10,    96, 13284,  1635,     9,  4244,   138,
             3,   524,   107,   127,   970,  7066,     9,     5,  3705,    52,
          3868,     3,  3781,   202,     3,  8607,    23,     3,  4031,    20,
            17,     9,     5,  2740,  7466,   144,     3,  4031,     3,    52,
          1024,  4244,    23,     5,  2740,     3,  1258,  1024,     6,     3,
          3781,     9,  4244,    23,     6,     8,    15,   157,  4244,    23,
            58,    41,   196,   751,    31,    17, 23970,     5,   432,     9,
             9,   107,   133,   103,  2891,    19,   132,    16,    82, 15126,
            61,   121,   970,  7066,   152,     9,    47,  4792,    16,    46,
          6326,    16,     3,     9,     1]])

In [13]:
after_summarize = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [14]:
print("After text summarization:")
after_summarize

After text summarization:


'a telephonic conversation with the army officer was started with a local. the officer replied: "Humara haal chhor Dujana. Surrender kyun nahi kar deta. Tu galat kar rha hai. Tu kaha, kya hai, theek hai? (I won\'t surrender. Allaah would do whatever is there in my fate)" Dujanana was killed in an encounter in a'

## Using Pipeline

In [15]:
from transformers import pipeline

In [16]:
summarizer = pipeline("summarization", model="t5-small", tokenizer="t5-small")

Device set to use cuda:0


In [17]:
context = """
Elon Musk, a visionary entrepreneur, is a name synonymous with groundbreaking innovations that transcend industries. Born in South Africa in 1971, Musk embarked on a journey that would lead him to revolutionize the realms of technology, space exploration, and sustainable energy.

One of Musk's most notable contributions lies in the electric vehicle sector. He co-founded Tesla Motors in 2003, aiming to make electric cars a mainstream reality. With Tesla's sleek designs, impressive performance, and advancements in battery technology, Musk has not only disrupted the automotive industry but also accelerated the world's transition to sustainable transportation.

SpaceX, another brainchild of Musk, is at the forefront of space exploration. Musk's audacious goal to make life multiplanetary has driven SpaceX to develop reusable rockets and drastically reduce the cost of space travel. The successful landing of Falcon rockets and the launch of the Falcon Heavy have reshaped the space industry, making it more accessible and economical.
"""

In [18]:
summary = summarizer(context, max_length=150, min_length=50, do_sample=True, top_k=50, top_p=0.95)

In [19]:
summary[0]['summary_text']

'Elon Musk, a visionary entrepreneur, is a name synonymous with groundbreaking innovations that transcend industries . born in south africa in 1971, Musk embarked on a journey that would lead him to revolutionize the realms of technology, space exploration, and sustainable energy . he co-founded Tesla Motors in 2003, aiming to make electric cars a mainstream reality .'

In [20]:
print("Before:",len(context))
print("After:",len(summary[0]['summary_text']))

Before: 1042
After: 369


## Using GPU

In [21]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [22]:
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)

>With your model and data on the GPU, you can now proceed with training or inference as usual. Just make sure that any tensors you create or process during these steps are also moved to the GPU.

In [23]:
model = model.to(device)
tokenized_text = tokenized_text.to(device)

In [24]:
summary_ids = model.generate(tokenized_text,
                                    no_repeat_ngram_size=3,
                                    min_length=40,
                                    max_length=150,
                                    early_stopping=True)

In [25]:
summary_ids

tensor([[    0,     3,     9,     3,  1931,  9621,   447,  3634,    28,     8,
          9102,  5502,    47,   708,    28,     3,     9,   415,     3,     5,
             8,  5502, 18606,    10,    96, 13284,  1635,     9,  4244,   138,
             3,   524,   107,   127,   970,  7066,     9,     5,  3705,    52,
          3868,     3,  3781,   202,     3,  8607,    23,     3,  4031,    20,
            17,     9,     5,  2740,  7466,   144,     3,  4031,     3,    52,
          1024,  4244,    23,     5,  2740,     3,  1258,  1024,     6,     3,
          3781,     9,  4244,    23,     6,     8,    15,   157,  4244,    23,
            58,    41,   196,   751,    31,    17, 23970,     5,   432,     9,
             9,   107,   133,   103,  2891,    19,   132,    16,    82, 15126,
            61,   121,   970,  7066,   152,     9,    47,  4792,    16,    46,
          6326,    16,     3,     9,     1]], device='cuda:0')

In [26]:
after_summarize = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [27]:
after_summarize

'a telephonic conversation with the army officer was started with a local. the officer replied: "Humara haal chhor Dujana. Surrender kyun nahi kar deta. Tu galat kar rha hai. Tu kaha, kya hai, theek hai? (I won\'t surrender. Allaah would do whatever is there in my fate)" Dujanana was killed in an encounter in a'

## Release cache

While working with large language models, it is always a best pratice to clear cache when you run in Memory Error

In [28]:
torch.cuda.empty_cache()